**Name:** Hosam Ali

**ID:** 314855792

In [1]:
%reload_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds

Load The MNIST Dataset

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


Process Data

In [3]:
def normalize_img(image, label):
    """Normalizes images: uint8 -> float32."""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
        normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

Neural network

In [4]:
model =  tf.keras.models.Sequential()

layers = [
          tf.keras.layers.Flatten(input_shape=(28, 28, 1)),

          #1
          tf.keras.layers.Dense(92),
          tf.keras.layers.Activation('tanh'),
          tf.keras.layers.Dropout(0.1),
          tf.keras.layers.BatchNormalization(),


          #2
          tf.keras.layers.Dense(57),
          tf.keras.layers.Activation('tanh'),


          #3
          tf.keras.layers.Dense(85),
          tf.keras.layers.Activation('tanh'),
          tf.keras.layers.Dropout(0.1),
          tf.keras.layers.BatchNormalization(),


          #4
          tf.keras.layers.Dense(14),
          tf.keras.layers.Activation('tanh'),
          tf.keras.layers.BatchNormalization(),

]

for layer in layers:
    model.add(layer)

# Output layer
model.add( tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 92)                  │          72,220 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 92)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 92)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 92)                  │             368 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 57)                  │           5,301 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 57)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 85)                  │           4,930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 85)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 85)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 85)                  │             340 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 14)                  │           1,204 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 14)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 14)                  │              56 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │             150 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 84,569 (330.35 KB)

 Trainable params: 84,187 (328.86 KB)

 Non-trainable params: 382 (1.49 KB)

Model Fit

In [5]:
model.fit(ds_train,
          epochs=20,
          validation_data=ds_test)

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.6044 - sparse_categorical_accuracy: 0.8261 - val_loss: 0.1870 - val_sparse_categorical_accuracy: 0.9455
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.2348 - sparse_categorical_accuracy: 0.9305 - val_loss: 0.1578 - val_sparse_categorical_accuracy: 0.9555
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.1872 - sparse_categorical_accuracy: 0.9444 - val_loss: 0.1229 - val_sparse_categorical_accuracy: 0.9646
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.1679 - sparse_categorical_accuracy: 0.9513 - val_loss: 0.1098 - val_sparse_categorical_accuracy: 0.9697
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.1441 - sparse_categorical_accuracy: 0.9582 - val_loss: 0.1013 - val_sparse_categorical_accuracy: 0.9704
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.1334 - sparse_categorical_accuracy: 0.9609 - val_loss: 0.1103 - val_sparse_categorical_accurac